#### RAGアプリ

>アプリの概要

<img src="pic/rag.png">

>目的

- RAGの仕組みの理解
    - ベクトル検索（Embedding）
- 基本的なRAGの実装
- 特定のWebページの情報を元に必要な回答を引き出すRAGの実装

>RAGのイメージ

<img src="pic/rag1.png">



>RAGにおけるデータソース

<img src="pic/rag3.png">

<img src="pic/rag6.png">

>キーワード検索とベクトル検索

<img src="pic/rag5.png">



>ベクトル検索

<img src="pic/rag2.png">
<img src="pic/rag4.png">

>RAGとEmbedding

・ Embedding：テキストを高次元ベクトル化すること。

・ RAG：データソースから取得した情報をベースにLLMが回答を生成する手法。
 - 両者は全く別ものであるが、組み合わせることで、「ベクトルデータベースを利用したRAG」を構築することが可能。
 - ベクトルデータベースを利用すれば、Embeddingによって高次元ベクトル化されたデータを格納することで、類似度検索エンジンを構築することができる。
 - RAGにおいて、ベクトルデータベースを利用するやり方が現在ポピュラーになっている。


>RAGのEmbeddingの作業の流れ

1. テキストのEmbedding
 - RAGでは、まず、データベース内の文書やユーザーからのクエリを数値のベクトルに変換します。
 - この変換プロセスによって、テキストデータが機械学習モデルが解釈しやすい形式になります。

2. 関連情報の検索
 - Embeddingされたクエリベクトルを使用して、データベース内から関連する文書や情報を検索します。
 - このとき、クエリベクトルと文書ベクトル間の類似性を計算し、最も関連性の高い文書を特定します。

3. テキスト生成のための情報の統合
 - 検索された文書や情報は、読みやすい形式に変換された後、元のユーザーのクエリやプロンプトと組み合わせされます。
 - この強化されたプロンプトが、LLMに入力され、テキスト生成の基盤となります。

4. テキストの生成
 - LLMは、強化されたプロンプトを基にして、関連性の高い回答やテキストを生成します。
 - このプロセスにより、ユーザーの質問に対する詳細で関連性の高い回答を提供することが可能になります。

>動作を構成してる部品の紹介（APIやライブラリ）

API
- openAIのchatGPT
    - 目的：AIのモデルを使う。
    - 参考：https://platform.openai.com/docs/api-reference/introduction?lang=python

- openAIのEmbedding
    - 目的：言語情報をベクトル化し、効果的に機械学習に利用する
    - 参考：https://weel.co.jp/media/tech/text-embedding-3/

ライブラリ
- streamlit
    - 目的：特定のWebページの情報を元に必要な回答を引き出すためのUIを表示するためのフロントエンドを生成する
- openai
    - 目的：openAIのchatGPT、Embeddingを活用するためのライブラリ
- requests
    - 目的：HTTPリクエストを送信するためのライブラリ。WebページやAPIからデータを取得する際に使用される。
- BeautifulSoup
    - 目的：HTMLやXMLを解析し、必要な情報を抽出するためのライブラリ。requestsで取得したHTMLデータを構造化し、検索・抽出しやすくするために使用される。
- cosine_similarity
    - 目的：ベクトル間のコサイン類似度を計算するためのライブラリ。2つのベクトルがどれだけ類似しているかを評価するために使用される。

>ライブラリのインストール

ライブラリ
- streamlit
    - pip install streamlit
- openai
    - pip install openai
- requests
    - pip install requests
- BeautifulSoup
    - pip install beautifulsoup4
- cosine_similarity
    - pip install scikit-learn



In [ ]:
import streamlit as st # フロントエンドを扱うstreamlitの機能をインポート
from openai import OpenAI  # OpenAIライブラリをインポートして、OpenAIのAPIとやり取りする
import os                  # 環境変数を管理するためにosをインポート
import requests  # Webリクエストを行うためのライブラリ
from bs4 import BeautifulSoup  # HTMLを解析するためのBeautifulSoupライブラリをインポート
from sklearn.metrics.pairwise import cosine_similarity  # cosine_similarityをインポートして類似度を計算

# OpenAI APIキーを環境変数として設定
os.environ["OPENAI_API_KEY"] = 'ご自身のOPENAI_API_KEYを入力'

# OpenAIクライアントを初期化
client = OpenAI()

def vectorize_text(text):
    """
    テキスト文字列を、OpenAIの埋め込みモデルを使用して対応するベクトルに変換する関数。
    """
    response = client.embeddings.create(
        input=text,                      # 埋め込みに変換するテキスト
        model="text-embedding-3-small"   # 使用する埋め込みモデルを指定
    )
    return response.data[0].embedding    # レスポンスから埋め込みベクトルを抽出して返す

# 質問を定義、この質問に最も類似した文書を見つけたい
question = "2023年の第1事業部の売上はどのくらい？"

# 質問と比較するための文書のリストを定義
documents = [
    "2023年上期売上200億円、下期売上300億円",
    "2023年第1事業部売上300億円、第2事業部売上150億円、第3事業部売上50億円",
    "2024年は全社で1000億円の売上を目指す"
]

# リスト内の各文書に対して埋め込みベクトルを生成
vectors = [vectorize_text(doc) for doc in documents]

# 質問に対して埋め込みベクトルを生成
question_vector = vectorize_text(question)


In [ ]:

max_similarity = 0  # 最大の類似度を格納するための変数を初期化
most_similar_index = 0  # 最も類似している文書のインデックスを格納するための変数を初期化

# すべての文書ベクトルを順番にチェック
for index, vector in enumerate(vectors):
    # 質問ベクトルと現在の文書ベクトルとのコサイン類似度を計算
    similarity = cosine_similarity([question_vector], [vector])[0][0]
    
    # 現在の文書とその類似度を表示
    print(documents[index], ":", similarity)
    
    # 計算された類似度がこれまでの最大類似度より大きい場合
    if similarity > max_similarity:
        max_similarity = similarity  # 最大類似度を更新
        most_similar_index = index   # 最も類似している文書のインデックスを更新

# 最も類似している文書を表示
print(documents[most_similar_index])


In [ ]:
# ユーザーの質問と最も類似している文書を使ってプロンプトを作成
prompt = f'''以下の質問に以下の情報をベースにして答えて下さい。
[ユーザーの質問]
{question}

[提供された情報]
{documents[most_similar_index]}
'''

# GPT-4o-miniモデルを使用して、ユーザーの質問に基づいて回答を生成
response = client.chat.completions.create(
    model="gpt-4o-mini",  # 使用するモデルを指定
    messages=[
        {"role": "system", "content": "You are a helpful assistant."},  # システムメッセージでアシスタントの役割を設定
        {"role": "user", "content": prompt}  # ユーザーの質問と提供された情報を含むプロンプトを渡す
    ],
    max_tokens=200  # レスポンスの最大トークン数を設定
)

# モデルが生成した回答を表示
print(response.choices[0].message.content)

## 特定のURLの情報にRAGを使ってアクセスしてチャットボットから回答を得る

In [ ]:
import requests  # Webリクエストを行うためのライブラリをインポート
from bs4 import BeautifulSoup  # HTMLを解析するためのBeautifulSoupライブラリをインポート

# 解析するためのWebページのURLを設定
url = "https://tech0-jp.com/#program"

# 指定されたURLに対してHTTP GETリクエストを送信し、レスポンスを取得
response = requests.get(url)

# レスポンスのHTMLコンテンツをBeautifulSoupを使って解析
soup = BeautifulSoup(response.text, "html.parser")

# ページ内のすべての<div>要素を取得
text_nodes = soup.find_all("div")

# すべての<div>要素のテキストを連結し、タブや改行を削除
joined_text = "".join(t.text.replace("\t", "").replace("\n", "").replace(" ", "") for t in text_nodes)

In [ ]:
joined_text

In [ ]:
len(joined_text)

In [ ]:
chunk_size = 400  # 各チャンクのサイズを400文字に設定
overlap = 50  # 各チャンク間で重複する文字数を50文字に設定
chunks = []  # テキストチャンクを格納するためのリストを初期化
start = 0  # チャンクの開始位置を初期化

# テキストの長さがチャンクサイズを超えるまで繰り返し処理を行う
while start + chunk_size <= len(joined_text):
    # 現在の開始位置からチャンクサイズ分のテキストを切り出してリストに追加
    chunks.append(joined_text[start:start + chunk_size])
    # 次のチャンクの開始位置を更新（重複部分を除く）
    start += (chunk_size - overlap)

# 残りのテキストがある場合、それを最後のチャンクとしてリストに追加
if start < len(joined_text):
    chunks.append(joined_text[-chunk_size:])

In [ ]:
# 各チャンクを順番に処理するためにループを設定
for i, chunk in enumerate(chunks):
    # チャンクの番号と、そのチャンクの最初の50文字と最後の50文字を表示
    # チャンク番号は1から始めるようにiに1を足している
    print(f"Chunk {i+1} : {chunk[:50]}...{chunk[-50:]}")

## 特定のURLの情報を元に作ったチャンクをベースにRAG

In [ ]:
import streamlit as st # フロントエンドを扱うstreamlitの機能をインポート
import requests  # Webリクエストを行うためのライブラリ
from bs4 import BeautifulSoup  # HTMLを解析するためのライブラリ
from openai import OpenAI  # OpenAIのAPIを利用するためのライブラリ
import os  # 環境変数の管理のためのライブラリ
from sklearn.metrics.pairwise import cosine_similarity  # コサイン類似度を計算するための関数

# OpenAI APIキーを環境変数に設定
os.environ["OPENAI_API_KEY"] = 'ご自身のOPENAI_API_KEYを入力'
client = OpenAI(api_key=os.getenv("OPENAI_API_KEY"))  # OpenAIクライアントを初期化

# 指定されたURLから記事をスクレイピングし、テキストを返す関数
def scrape_article(url):
    response = requests.get(url)  # 指定されたURLにGETリクエストを送信
    soup = BeautifulSoup(response.text, "html.parser")  # 取得したHTMLを解析
    text_nodes = soup.find_all("div")  # すべての<div>タグを取得
    joined_text = "".join(t.text.replace("\t", "").replace("\n", "") for t in text_nodes)  # テキストを連結し、タブや改行を削除
    return joined_text  # 連結されたテキストを返す

# テキストをチャンクに分割する関数
def chunk_text(text, chunk_size, overlap):
    chunks = []  # チャンクを格納するリストを初期化
    start = 0  # チャンクの開始位置を初期化
    while start + chunk_size <= len(text):
        chunks.append(text[start:start + chunk_size])  # チャンクをリストに追加
        start += (chunk_size - overlap)  # 次のチャンクの開始位置を設定
    if start < len(text):
        chunks.append(text[-chunk_size:])  # 残りのテキストがある場合は最後のチャンクとして追加
    return chunks  # 分割されたチャンクのリストを返す

# テキストをベクトル化する関数
def vectorize_text(text):
    response = client.embeddings.create(
        input=text,
        model="text-embedding-ada-002"  # 正しいモデル名を指定
    )
    return response.data[0].embedding  # 生成された埋め込みベクトルを返す

# 質問ベクトルと文書ベクトルを比較して最も類似した文書を見つける関数
def find_most_similar(question_vector, vectors, documents):
    similarities = []  # 類似度を格納するリストを初期化
    for index, vector in enumerate(vectors):
        similarity = cosine_similarity([question_vector], [vector])[0][0]  # コサイン類似度を計算
        similarities.append([similarity, index])  # 類似度と文書のインデックスをリストに追加
    similarities.sort(reverse=True, key=lambda x: x[0])  # 類似度の降順でソート
    top_documents = [documents[index] for similarity, index in similarities[:2]]  # 上位2つの文書を取得
    return top_documents  # 最も類似した文書を返す

# 質問を基にGPT-4モデルで回答を生成する関数
def ask_question(question, context):
    # 質問と文脈情報を基にプロンプトを作成
    prompt = f'''以下の質問に以下の情報をベースにして答えて下さい。
    [ユーザーの質問]
    {question}

    [情報]
    {context}
    '''
    print(prompt)  # 作成したプロンプトを表示（デバッグ用）

    # OpenAIのチャットモデルにリクエストを送信して回答を生成
    response = client.chat.completions.create(
        model="gpt-4o-mini",  # 正しいモデル名を指定
        messages=[
            {"role": "system", "content": "You are a helpful assistant."},  # システムメッセージでアシスタントの役割を設定
            {"role": "user", "content": prompt}  # ユーザーからの質問と文脈情報を含むプロンプトを渡す
        ],
        max_tokens=400  # レスポンスの最大トークン数を設定
    )
    return response.choices[0].message.content  # 生成された回答のテキストを返す

# 記事のURLとテキストチャンク分割のパラメータを設定
url = "https://tech0-jp.com/#program"
chunk_size = 400
overlap = 50

# 指定されたURLから文章をスクレイピングし、テキストチャンクに分割
article_text = scrape_article(url)
text_chunks = chunk_text(article_text, chunk_size, overlap)

# 各テキストチャンクをベクトル化
vectors = [vectorize_text(doc) for doc in text_chunks]

# 質問をベクトル化し、最も類似した文書を検索
question = "Step2ではどのようなことが身につけられますか？"
question_vector = vectorize_text(question)
similar_document = find_most_similar(question_vector, vectors, text_chunks)

# 質問を基に回答を生成
answer = ask_question(question, similar_document)
print(answer)  # 生成された回答を表示